In [2]:
%pip install opik openai comet_ml litellm --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.5/303.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 710.6/710.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 980.3/980.3 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 73.1 MB/s eta 0:00:00


In [3]:
import opik
from opik import Opik, track
from opik.evaluation import evaluate
from opik.evaluation.metrics import (Hallucination, AnswerRelevance)
from opik.integrations.openai import track_openai
import openai
import os
from datetime import datetime
from getpass import getpass
import litellm
from litellm.integrations.opik.opik import OpikLogger
from opik.opik_context import get_current_span_data

opik_logger = OpikLogger()
# In order to log LiteLLM traces to Opik, you will need to set the Opik callback
litellm.callbacks = [opik_logger]

# Define project name to enable tracing. Project name already logged into OPIK
os.environ["OPIK_PROJECT_NAME"] = "food_chatbot_eval"

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [4]:
# opik configs
if "OPIK_API_KEY" not in os.environ:
    os.environ["OPIK_API_KEY"] = getpass("Enter your Opik API key: ")

opik.configure()

Enter your Opik API key: ··········
Do you want to use "bluemusk" workspace? (Y/n)y


OPIK: Configuration saved to file: /root/.opik.config


In [5]:
# Hugging Face Configs to access meta-llama-3.2 model
if "HF_TOKEN" not in os.environ:
  os.environ["HF_TOKEN"] = getpass("Enter your Hugging Face Key: ")

Enter your Hugging Face Key: ··········


In [6]:
# get Opik client for dataset and Model from huggingFace
client = Opik()

MODEL = "huggingface/meta-llama/Llama-3.2-3B-Instruct"

# Templates & Prompts

In [7]:
# define menu items for the food chatbot
menu_items = """
Menu: Kids Menu
Food Item: Mini Cheeseburger
Price: $6.99
Vegan: N
Popularity: 4/5
Included: Mini beef patty, cheese, lettuce, tomato, and fries.

Menu: Appetizers
Food Item: Loaded Potato Skins
Price: $8.99"
Vegan: N
Popularity: 3/5
Included: Crispy potato skins filled with cheese, bacon bits, and served with sour cream.

Menu: Appetizers
Food Item: Bruschetta
Price: $7.99
Vegan: Y
Popularity: 4/5
Included: Toasted baguette slices topped with fresh tomatoes, basil, garlic, and balsamic glaze.

Menu: Main Menu
Food Item: Grilled Chicken Caesar Salad
Price: $12.99
Vegan: N
Popularity: 4/5
Included: Grilled chicken breast, romaine lettuce, Parmesan cheese, croutons, and Caesar dressing.

Menu: Main Menu
Food Item: Classic Cheese Pizza
Price: $10.99
Vegan: N
Popularity: 5/5
Included: Thin-crust pizza topped with tomato sauce, mozzarella cheese, and fresh basil.

Menu: Main Menu
Food Item: Spaghetti Bolognese
Price: $14.99
Vegan: N
Popularity: 4/5
Included: Pasta tossed in a savory meat sauce made with ground beef, tomatoes, onions, and herbs.

Menu: Vegan Options
Food Item: Veggie Wrap
Price: $9.99
Vegan: Y
Popularity: 3/5
Included: Grilled vegetables, hummus, mixed greens, and a wrap served with a side of sweet potato fries.

Menu: Vegan Options
Food Item: Vegan Beyond Burger
Price: $11.99
Vegan: Y
Popularity: 4/5
Included: Plant-based patty, vegan cheese, lettuce, tomato, onion, and a choice of regular or sweet potato fries.

Menu: Desserts
Food Item: Chocolate Lava Cake
Price: $6.99
Vegan: N
Popularity: 5/5
Included: Warm chocolate cake with a gooey molten center, served with vanilla ice cream.

Menu: Desserts
Food Item: Fresh Berry Parfait
Price: $5.99
Vegan: Y
Popularity: 4/5
Included: Layers of mixed berries, granola, and vegan coconut yogurt.
"""

In [8]:
# function call of llama3 using litellm
def generate_response(prompt):
  response = litellm.completion(
      model=MODEL,
      messages=[
          {"role":"system", "content":"You are a helpful assistant."},
          {"role":"user", "content":prompt}
      ]
  )

  return response.choices[0].message.content

# LLM Call Steps

In [9]:
@track
def reasoning_step(user_query, menu_items):
    prompt_template = """
    Your task is to answer questions factually about a food menu, provided below and delimited by +++++. The user request is provided here: {request}

    Step 1: The first step is to check if the user is asking a question related to any type of food (even if that food item is not on the menu). If the question is about any type of food, we move on to Step 2 and ignore the rest of Step 1. If the question is not about food, then we send a response: "Sorry! I cannot help with that. Please let me know if you have a question about our food menu."

    Step 2: In this step, we check that the user question is relevant to any of the items on the food menu. You should check that the food item exists in our menu first. If it doesn't exist then send a kind response to the user that the item doesn't exist in our menu and then include a list of available but similar food items without any other details (e.g., price). The food items available are provided below and delimited by +++++:

    +++++
    {menu_items}
    +++++

    Step 3: If the item exists in our food menu and the user is requesting specific information, provide that relevant information to the user using the food menu. Make sure to use a friendly tone and keep the response concise.

    Perform the following reasoning steps to send a response to the user:
    Step 1: <Step 1 reasoning>
    Step 2: <Step 2 reasoning>
    Response to the user (only output the final response): <response to user>
    """

    prompt = prompt_template.format(request=user_query, menu_items=menu_items)
    response = generate_response(prompt)
    return response

In [10]:
@track
def extraction_step(reasoning):
    prompt_template = """
    Extract the final response from delimited by ###.

    ###
    {reasoning}.
    ###

    Only output what comes after "Response to the user:".
    """

    prompt = prompt_template.format(reasoning=reasoning)
    response = generate_response(prompt)
    return response

In [11]:
@track
def refinement_step(final_response):
    prompt_template = """
    Perform the following refinement steps on the final output delimited by ###.

    1). Shorten the text to one sentence
    2). Use a friendly tone

    ###
    {final_response}
    ###
    """

    prompt = prompt_template.format(final_response=final_response)
    response = generate_response(prompt)
    return response

In [12]:
@track
def verification_step(user_question, refined_response, menu_items):
    prompt_template = """
    Your task is to check that the refined response (delimited by ###) is providing a factual response based on the user question (delimited by @@@) and the menu below (delimited by +++++). If yes, just output the refined response in its original form (without the delimiters). If no, then make the correction to the response and return the new response only.

    User question: @@@ {user_question} @@@

    Refined response: ### {refined_response} ###

    +++++
    {menu_items}
    +++++
    """

    prompt = prompt_template.format(user_question=user_question, refined_response=refined_response, menu_items=menu_items)
    response = generate_response(prompt)
    return response

# LLM Application
Adding tracking to your LLM application allows you to have full visibility into each evaluation run.

In [13]:
@track
def generate_chatbot_response(user_query, menu_items):
    reasoning = reasoning_step(user_query, menu_items)
    extraction = extraction_step(reasoning)
    refinement = refinement_step(extraction)
    verification = verification_step(user_query, refinement, menu_items)
    return verification

@track
def chatbot_application(input: str) -> str:
    response = generate_chatbot_response(input, menu_items)
    return response

# Define the Evaluation Task
The evaluation task takes in as an input a dataset item and needs to return a dictionary with keys that match the parameters expected by the metrics you are using.

In [14]:
# Define the evaluation task
def evaluation_task(x):
    return {
        "input": x['question'],
        "output": chatbot_application(x['question']),
        "context": menu_items,
        "reference": x['response']
    }

# Dataset
To retrieve an existing dataset, use client.get_dataset(), or to create a new dataset, use client.get_or_create_dataset().

In [15]:
# Create or retrieve the dataset
client = Opik()
dataset = client.get_or_create_dataset(name="foodchatbot_eval")

# Choose Evaluation Metrics
Opik provides a set of built-in evaluation metrics, including heuristic metrics and LLMs-as-a-judge.

Note that each metric expects the data in a certain format, you will need to ensure that the task you have defined in step 1. returns the data in the correct format.

In [16]:
# Define the metrics
metrics = [Hallucination(), AnswerRelevance()]

# experiment_name
experiment_name = MODEL + "_" + dataset.name + "_" + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Run the evaluation

In [17]:
# run evaluation
evaluation = evaluate(
    experiment_name=experiment_name,
    dataset=dataset,
    task=evaluation_task,
    scoring_metrics=metrics,
    experiment_config={
        "model": MODEL
    }
)

Evaluation:   0%|          | 0/56 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Streaming output truncated to the last 5000 lines.
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Client error '403 Forbidden' for url 'https://www.comet.com/opik/api/v1/private/traces/batch'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
ERROR:LiteLLM:OpikLogger failed to send batch - Client error '403 Forbidden' for url 'https://www.comet.com/opik/api/v1/private/traces/batch'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/litellm/integrations/opik/opik.py", line 102, in _sync_send
    response = self.sync_httpx_client.post(
  File "/usr/local/lib/python3.10/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 528, in post
    raise e
  File "/usr/local/lib/python3.10/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 509, in post
    response.raise_for_sta

╭─ foodchatbot_eval (56 samples) ─────────────────╮
│                                                 │
│ Total time:        00:00:43                     │
│ Number of samples: 56                           │
│                                                 │
│ hallucination_metric: None (avg) - 56 failed    │
│ answer_relevance_metric: None (avg) - 56 failed │
│                                                 │
╰─────────────────────────────────────────────────╯

Uploading results to Opik ...

04:29:54 - LiteLLM:ERROR: opik.py:111 - OpikLogger failed to send batch - Client error '403 Forbidden' for url 'https://www.comet.com/opik/api/v1/private/spans/batch'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/litellm/integrations/opik/opik.py", line 102, in _sync_send
    response = self.sync_httpx_client.post(
  File "/usr/local/lib/python3.10/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 528, in post
    raise e
  File "/usr/local/lib/python3.10/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 509, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.10/dist-packages/httpx/_models.py", line 763, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Client error '403 Forbidden' for url 'https://www.comet.com/opik/api/v1/private/spans/batch'
For more info

View the results ]8;id=166696;https://www.comet.com/opik/bluemusk/experiments/01941b11-dda6-7256-aeca-79831c77b744/compare?experiments=%5B%220194442c-6f24-7375-b444-3b307400da22%22%5D\in your Opik dashboard]8;;\.

04:29:54 - LiteLLM:ERROR: opik.py:111 - OpikLogger failed to send batch - Client error '403 Forbidden' for url 'https://www.comet.com/opik/api/v1/private/spans/batch'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/litellm/integrations/opik/opik.py", line 102, in _sync_send
    response = self.sync_httpx_client.post(
  File "/usr/local/lib/python3.10/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 528, in post
    raise e
  File "/usr/local/lib/python3.10/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 509, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.10/dist-packages/httpx/_models.py", line 763, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Client error '403 Forbidden' for url 'https://www.comet.com/opik/api/v1/private/spans/batch'
For more info

In [22]:
# metrics contained in OPIK
import opik.evaluation.metrics as metrics
print(dir(metrics))


['AnswerRelevance', 'BaseMetric', 'Contains', 'ContextPrecision', 'ContextRecall', 'Equals', 'GEval', 'Hallucination', 'IsJson', 'LevenshteinRatio', 'MetricComputationError', 'Moderation', 'RegexMatch', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'arguments_helpers', 'base_metric', 'exceptions', 'heuristics', 'llm_judges', 'score_result']
